In [1]:
%load_ext autoreload
%autoreload 2
#---------- Library Imports ----------
import torch
import cv2
import numpy as np
from PIL import Image
from torchvision import transforms
import os
import sequential_creator
import glob
from torchmetrics import MeanAbsolutePercentageError as MAPE
import matplotlib as mpl
import re
import json
from tqdm import tqdm
import pytorch_lightning as pl
from torch.utils.data import DataLoader

c:\Users\Ulric\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%reload_ext autoreload

In [3]:
#---------- Personal python files imports ----------
import load_lettuce_dataset
import data_augmentation
import model_and_training_files
import lettuce_dataset

In [4]:
#---------- Hyperparameters ----------
batch_size = 16
learning_rate = 1e-3

In [5]:
#---------- Other parameters ----------
augmented_dataset_size = 1000 # Size of the augmented dataset, so if the original dataset contained 103 images, they would be augmented and made into 2000 images
                              # One thing to note is that if this number is much bigger than the size of the original dataset, then they would most likely end up being duplicates, since there is not that many augmentations implemented at the moment

In [6]:
#---------- Load Lettuce Dataset ----------
rgb_list, depth_list, fresh_weight_list, dry_weight_list = load_lettuce_dataset.load_all_images()

100%|██████████| 341/341 [02:17<00:00,  2.48it/s]


In [7]:
#---------- Augment Lettuce Dataset ----------
depth_img_aug, rgb_imgs_aug, fresh_weight_GT, dry_weight_GT = data_augmentation.augment_data(rgb_images=rgb_list, depth_images=depth_list, fresh_weight_GT=fresh_weight_list, dry_weight_GT=dry_weight_list, amount_of_augmentated_images=augmented_dataset_size)

Augmenting RGB Images


100%|██████████| 1000/1000 [09:43<00:00,  1.71it/s]

Amount of augmented images 1000


In [ ]:
#---------- Create data loaders ----------
# Concatenate the depth and rgb images
full_dataset = []
full_dataset_labels = []
for i in range(augmented_dataset_size):
    full_dataset.append([depth_img_aug[i], rgb_imgs_aug[i]])
    full_dataset_labels.append([fresh_weight_GT[i], dry_weight_GT[i]])

print("Size of images in the dataset: ", np.array(full_dataset[0][0]).shape)

# Define the dataset
dataset = lettuce_dataset.LettuceDataset(full_dataset, full_dataset_labels)

# Split the dataset in train, validation and test
splitted_data = lettuce_dataset.data_splittage(augmented_dataset_size, [75, 12.5, 12.5])
print("train, validation, test = ", splitted_data)

train_set, validation_set, test_set = torch.utils.data.random_split(dataset, splitted_data)
#train_set, validation_set, test_set = torch.utils.data.random_split(dataset, [7500, 1250, 1250])

# Create dataloaders for train, validation and test data
train_loader = DataLoader(dataset = train_set, batch_size = batch_size, shuffle = True)
validation_loader = DataLoader(dataset = validation_set, batch_size = batch_size, shuffle = True)
test_loader = DataLoader(dataset = test_set, batch_size = batch_size, shuffle = True)

Size of images in the dataset:  (3, 224, 224)
train, validation, test =  [750, 125, 125]


In [9]:
#---------- Model Definition ----------
# Load ResNet Backbone (Pretrained Model)
resnet_version = ['resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152']
# We can reuse the same resnet model since weights are not updated and both RGB and Depth are same format
resnet_model_RGB_and_depth = torch.hub.load('pytorch/vision:v0.10.0', resnet_version[3], weights="ResNet101_Weights.DEFAULT")
resnet_model_RGB_and_depth.eval() # We don't want the resnet to update (Sets a flag, kind of a switch to turn off gradient computation)

# Create Regression Head
head_input_neurons = 2000 # 1000 output feature vector from RGB resnet and 1000 from depth resnet
head_hidden = [1000, 500, 250]
head_output_neurons = 2
head_activation = torch.nn.ReLU()
regression_head = sequential_creator.make_model(input=head_input_neurons, hidden=head_hidden, output=head_output_neurons, activation=head_activation)

print(regression_head)

Using cache found in C:\Users\Ulric/.cache\torch\hub\pytorch_vision_v0.10.0


Sequential(
  (0): Linear(in_features=2000, out_features=1000, bias=True)
  (1): ReLU()
  (2): Linear(in_features=1000, out_features=500, bias=True)
  (3): ReLU()
  (4): Linear(in_features=500, out_features=250, bias=True)
  (5): ReLU()
  (6): Linear(in_features=250, out_features=2, bias=True)
  (7): ReLU()
)


In [12]:
#---------- Training ----------
print("Cuda available = ", torch.cuda.is_available())

# Define the model with Pytorch Lightning
model = model_and_training_files.BiomassModel(regression_head=regression_head, 
                                                resnet_model_RGB_and_depth=resnet_model_RGB_and_depth,
                                                lr=learning_rate,
                                                )

trainer = model_and_training_files.get_trainer()    # gets the trainer (which is a class that takes the model and dataset)
trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=validation_loader) # Train the model
trainer.test(model, dataloaders=test_loader)

#---------- Save the weights ----------
torch.save(model, "saved_models/best_weights_V1.plk") # Saves the regression head

Auto select gpus: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Cuda available =  True


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type       | Params
--------------------------------------
0 | layers | Sequential | 2.6 M 
1 | Resnet | ResNet     | 44.5 M
--------------------------------------
47.2 M    Trainable params
0         Non-trainable params
47.2 M    Total params
188.706   Total estimated model params size (MB)


Epoch 1:  36%|███▋      | 20/55 [00:14<00:25,  1.39it/s, loss=0.24, v_num=5] 

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 8/8 [00:04<00:00,  1.62it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss            7.060391902923584
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


AttributeError: Can't pickle local object 'EvaluationLoop.advance.<locals>.batch_to_device'